# Principal Component Analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

## Datensatz

Wir betrachten wieder eine Datensatz mit handgeschriebenen Ziffern, diesmal allerdings in etwas höherer Auflösung (28x28 statt 8x8). Außerdem haben wir wesentlich mehr Datensätze zur Verfügung.

Die 28x28 Pixel ergeben 784 Features, d.h. unser Ausgangsraum ist $\mathbb{R}^{784}$.

In [ ]:
train = pd.read_csv('./data/mnist/train.csv')
target = train.label
train = train.drop("label",axis=1)
m,n = train.shape
print(f'Wir haben {m} Datenpunkte mit {n} Features')
print('Ein Beispielbild:')
fig, ax = plt.subplots(1,1, figsize=(8,8))
ax.imshow(np.asarray(train.iloc[1]).reshape(28,28), cmap='gray');

## Aufbereitung

Wir standardisieren die Features durch Entfernen des Mittelwerts und Skalierung auf Einheitsvarianz.

Der Standardwert einer Stichprobe x wird berechnet als:

$$z=\frac{x-\mu}{\sigma}$$

wobei $\mu$ der Mittelwert der Trainingsstichproben und $\sigma$ die Standardabweichung der Trainingsstichproben ist.

Zentrierung und Skalierung erfolgen unabhängig für jedes Merkmal, indem die relevanten Statistiken über die Stichproben im Trainingsset berechnet werden. Mittelwert und Standardabweichung werden dann gespeichert, um bei späteren Daten mittels Transformation verwendet zu werden.

Die Standardisierung eines Datensatzes ist eine häufige Anforderung für viele Schätzer des maschinellen Lernens: Sie könnten sich schlecht verhalten, wenn die einzelnen Merkmale nicht mehr oder weniger wie normalverteilte Standarddaten aussehen (z.B. Gaußsch mit Mittelwert 0 und Einheitsvarianz).

Beispielsweise gehen viele Elemente, die in der Cost-Function eines Lernalgorithmus verwendet werden (wie die L1- und L2-Regularisierer von linearen Modellen) davon aus, dass alle Merkmale um 0 zentriert sind und Varianzen in gleicher Größenordnung haben. Wenn ein Merkmal eine Varianz hat, die um Größenordnungen größer ist als andere, könnte es die Zielfunktion dominieren und den Schätzer unfähig machen, von anderen Merkmalen zu lernen.

In [ ]:
from sklearn.preprocessing import StandardScaler
X = train
scaler = StandardScaler().fit(X)
print(f'Skalierer: {scaler}')
X_std = scaler.transform(X)
X_std.shape

## Eigenwertzerlegung

Die Eigenvektoren und Eigenwerte einer Kovarianz- Matrix stellen den "Kern" einer PCA dar: 
Die Eigenvektoren (Hauptkomponenten/Principal Components) bestimmen die Richtungen
des neuen Featureraums, und die Eigenwerte bestimmen ihre Größe. 

Mit anderen Worten: Die Eigenwerte erklären die Varianz der Daten entlang der Achsen (Eigenvektoren) des neuen Merkmals.

## Berechne die Kovarianz Matrix

Der klassische Ansatz der PCA besteht darin, die Eigenwertzerlegung auf der Kovarianzmatrix $Cov(X)$ durchzuführen, die eine n×n-Matrix ist, in der jedes Element die Kovarianz zwischen
zwei Merkmalen repräsentiert. 

Die Kovarianz zwischen zwei Merkmalen wird wie folgt berechnet:

$$\sigma_{jk}=\frac{1}{n-1}\sum\limits_{i=1}^n{(x_i^{(j)}-\mu_j)(x_i^{(k)}-\mu_k)}$$

In Matrizenform geschrieben ist das:

$$Cov(X)=\frac{1}{n-1}((X-\mu)^T (X-\mu))$$

mit $\mu=\frac{1}{n}\sum\limits_{i=1}^n x^{(i)}$.

Die Kovarianzmatrix drückt die paarweisen Abhängigkeiten der Features untereinander aus, sie
ist also eine Verallgemeinerung der Korrelation in den n-dimensionalen Raum. 


In [ ]:
mean_vec = np.mean(X_std, axis=0)
cov = np.dot((X_std-mean_vec).T, X_std-mean_vec)/(m-1)
cov.shape

## Eigenwerte und -vektoren

Für einen Eigenvektor $v$ einer Matrix X gilt: $v\neq 0$ und es existiert ein $\lambda$, so dass $$Xv=\lambda v$$.

Die Eigenvektoren bilden eine Orthonormalbasis, d.h. wir können jeden Vektor als Linearkombination der Eigenvektoren schreiben.

Wenn wir die Eigenwerte unserer Kovarianzmatrix ermitteln, so spiegelt die Größe der Eigenwerte wieder, wie stark der Einfluss dieser Richtung auf unsere Daten ist. 

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(cov)
eig_vals.shape, eig_vecs.shape

Anteil an Varianz

In [ ]:
tot = sum(eig_vals)

var_exp = [(i/tot) for i in sorted(eig_vals, reverse=True)] # Individual explained variance
cum_var_exp = np.cumsum(var_exp) # Cumulative explained variance

## Plot der erklärten Varianz

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, figsize=(15,7))

axs[0].plot(list(range(n)), cum_var_exp, c='black', label='Kumulierte erklärte Varianz')
axs[0].plot(list(range(n)), var_exp, c='red', label='Erklärte Varianz')
axs[0].set_xlabel('Eigenvektor')
axs[0].axhline(1, c='grey')
axs[0].legend();

k = 250
axs[1].plot(list(range(k)), cum_var_exp[:k], c='black', label='Kumulierte erklärte Varianz')
axs[1].plot(list(range(k)), var_exp[:k], c='red', label='Erklärte Varianz')
axs[1].set_xlabel(f'Die ersten {k} Eigenvektoren')
axs[1].axhline(1, c='grey')
axs[1].axhline(cum_var_exp[k], c='grey', linestyle='dotted')
axs[1].text(0, cum_var_exp[k], f'{cum_var_exp[k]:.5f}')
axs[1].legend();



D.h. mit den ersten 250 der 784 Principal-Component-Features würden wir schon **über 90% der Varianz** abdecken.

## Scikit Learn's PCA

In [ ]:
pca = PCA().fit(X_std)

eigenvectors = pca.components_
cumul = np.cumsum(pca.explained_variance_ratio_)
fig, ax = plt.subplots(1, 1, figsize=(7,7))

ax.plot(list(range(n)), pca.explained_variance_ratio_, c='red', label='Erklärte Varianz')
ax.plot(list(range(n)), cumul, c='black', label='Kumulierte erklärte Varianz')
ax.set_xlabel('Eigenvektor')
ax.axhline(1, c='grey')
ax.legend();


## Plotte Eigenvektoren

Unsere Eigenvektoren sind Vektoren in $\mathbb{R}^{784}$. Gleichzeitig sind diese Vektoren aber ja auch jeweils die Pixel eines 28x28 Bildes.

D.h. wir können auch unsere Eigenvektoren als Bilder anschauen:

In [ ]:
n_row = 7
n_col = 7

fig, axs = plt.subplots(n_row, n_col, figsize=(20,3*n_row))

def plot_eigenvectors(row, col, eigenvector_nr):
    ax = axs[row, col]
    ax.imshow(eigenvectors[eigenvector_nr].reshape(28,28), cmap='gray') # Alternativ in Farbe: jet
    title_text = f'Eigenvektor {eigenvector_nr + 1}'
    ax.set_title(title_text, size=12)
    ax.set_xticks(())
    ax.set_yticks(())

offsets = [0, 10, 50, 100, 200, 500, n-n_col]

for i in list(range(n_row)):
    for j in range(n_col):
        plot_eigenvectors(i, j, offsets[i] + j)

plt.show()